In [7]:
import os
import gc
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Ridge, HuberRegressor
from sklearn.feature_selection import RFE



In [2]:
train_stacked = pd.read_csv('../oofs/kain-train-features-v0.1.2.csv', index_col=0)
test_stacked = pd.read_csv('../oofs/kain-test-features-v0.1.2.csv', index_col=0)

In [3]:
train_stacked.head()

,LGB_cat_0_7956629719199435_csv,LGB_cat_0_796965615822933_csv,LGB_cat_0_7971758465559547_csv,LGB_cat_0_7971922012487902_csv,LGB_cat_0_7975664475835779_csv,LGB_cat_0_7982459487910958_csv,LGB_cat_0_7985379016892897_csv,LGB_cat_0_7986254088147751_csv,LGB_cat_0_7987384223756171_csv,LGB_cat_0_7988757981190066_csv,...,TARGET.72,TARGET.73,TARGET.74,TARGET.75,TARGET.76,TARGET.77,TARGET.78,TARGET.79,TARGET.80,TARGET.81
0,0.263165,0.210463,0.254966,0.272226,0.242135,0.217145,0.170371,0.194649,0.213564,0.166478,...,0.673457,0.577123,0.615324,0.409131,0.339133,0.310050,0.292603,0.274747,0.229829,0.377313
1,0.028534,0.022802,0.021364,0.027252,0.025308,0.024883,0.021984,0.021119,0.023010,0.023200,...,0.103917,0.039187,0.074826,0.037902,0.000000,0.023711,0.018794,0.009405,0.019214,0.017564
2,0.044988,0.035597,0.037636,0.041098,0.039781,0.038767,0.047803,0.032391,0.034830,0.045556,...,0.098294,0.068005,0.105474,0.038344,0.055681,0.040151,0.030012,0.048608,0.041607,0.024725
3,0.028133,0.023076,0.025890,0.025638,0.025989,0.026900,0.027623,0.021785,0.027436,0.025201,...,0.035021,0.080346,0.060589,0.016141,0.048706,0.051309,0.037157,0.041925,0.030234,0.035943
4,0.061272,0.049731,0.047512,0.059441,0.059825,0.053356,0.059691,0.043630,0.050947,0.054172,...,0.129976,0.115208,0.147167,0.083344,0.075375,0.100184,0.088149,0.061036,0.056570,0.059047


In [4]:
test_stacked.head()

,LGB_cat_0_7956629719199435_csv,LGB_cat_0_796965615822933_csv,LGB_cat_0_7971758465559547_csv,LGB_cat_0_7971922012487902_csv,LGB_cat_0_7975664475835779_csv,LGB_cat_0_7982459487910958_csv,LGB_cat_0_7985379016892897_csv,LGB_cat_0_7986254088147751_csv,LGB_cat_0_7987384223756171_csv,LGB_cat_0_7988757981190066_csv,...,TARGET.72,TARGET.73,TARGET.74,TARGET.75,TARGET.76,TARGET.77,TARGET.78,TARGET.79,TARGET.80,TARGET.81
0,0.032046,0.034213,0.036182,0.030748,0.030909,0.030198,0.026424,0.033712,0.032925,0.027859,...,0.060607,0.085077,0.059713,0.015011,0.093184,0.112115,0.059259,0.055032,0.038632,0.055209
1,0.121353,0.118384,0.121478,0.120025,0.119085,0.120870,0.117530,0.117468,0.114038,0.112407,...,0.334951,0.236243,0.293807,0.141847,0.159056,0.220960,0.127498,0.134836,0.125285,0.151449
2,0.022355,0.023191,0.024048,0.023358,0.022430,0.024886,0.025242,0.021554,0.023749,0.025666,...,0.063316,0.042244,0.063244,0.044956,0.052548,0.016997,0.037535,0.042209,0.022144,0.022888
3,0.022851,0.026334,0.027283,0.026948,0.026514,0.028141,0.023710,0.027926,0.029214,0.020858,...,0.130619,0.080953,0.120677,0.047268,0.000000,0.043664,0.035744,0.067668,0.034352,0.034921
4,0.145823,0.131313,0.126619,0.139883,0.137116,0.142311,0.115298,0.122861,0.141986,0.114807,...,0.304686,0.332027,0.250013,0.103591,0.111472,0.136448,0.118943,0.143444,0.141113,0.118314


In [5]:
train = pd.read_csv('../../data/application_train.csv')
y = train['TARGET']

n_train = train.shape[0]

In [6]:
train_features = train_stacked.values
test_features = test_stacked.values

In [8]:
model = LogisticRegression(C=100, class_weight={0:1, 1:3},
                               fit_intercept=False, n_jobs=1, max_iter=10000)

In [9]:
selector = RFE(model, 100, step=2)
selector = selector.fit(train_features, y)

In [10]:
selected_features = [i for i, y in enumerate(selector.ranking_) if y == 1]

In [11]:
selected_features

[0,
 1,
 2,
 3,
 7,
 9,
 12,
 13,
 17,
 18,
 19,
 20,
 22,
 23,
 26,
 28,
 29,
 31,
 37,
 38,
 39,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 56,
 57,
 59,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 102,
 104,
 105,
 106,
 108,
 109,
 110,
 112,
 114,
 115,
 116,
 117,
 118,
 119,
 122,
 126,
 129,
 131,
 132,
 133,
 134,
 135,
 138,
 140,
 142,
 143,
 146,
 147,
 148,
 149,
 150,
 151,
 152]

In [12]:
train_features.shape

(307511, 154)

In [14]:
aucs = []
test_set = []
validation_set = []
print("\nModeling Stage")

kf = KFold(n_splits=5, random_state=1002, shuffle=True)
kf.get_n_splits(train_features)

n_bagged = 6

for train_index, test_index in kf.split(train_features):
    print("TRAIN: ", train_index, "TEST: ", test_index)

    X = train_features
    y_ = y.values
    x_train, x_test = X[train_index][:, selected_features], X[test_index][:, selected_features]
    y_train, y_test = y_[train_index], y_[test_index]
    
    oof_baggs = np.zeros([n_bagged , x_test.shape[0]])
    preds_baggs = np.zeros([n_bagged , test_features.shape[0]])
    
    for _it in range(1, n_bagged):
        
        model = LogisticRegression(C=0.1 * _it, class_weight={0: 1, 1: 5},
                               fit_intercept=False, n_jobs=1, max_iter=10000)
        model.fit(x_train, y_train)
        
        print(_it,' ' ,'Fold AUC :', roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]))
        oof_baggs[_it, :] = model.predict_proba(x_test)[:, 1]
        preds_baggs[_it, :] = model.predict_proba(test_features[:, selected_features])[:, 1]
        
    val_preds = pd.DataFrame(oof_baggs).T
    test_preds = pd.DataFrame(preds_baggs).T
    
    print('Fold AUC :', roc_auc_score(y_test, val_preds.rank(axis=0, method='min').mul(val_preds.shape[1] * [1 / val_preds.shape[1]]).sum(1) / val_preds.shape[0]))
    aucs.append(roc_auc_score(y_test, val_preds.rank(axis=0, method='min').mul(val_preds.shape[1] * [1 / val_preds.shape[1]]).sum(1) / val_preds.shape[0]))
   
    test_set.append(test_preds.rank(axis=0, method='min').mul(test_preds.shape[1] * [1 / test_preds.shape[1]]).sum(1) / test_preds.shape[0])
    gc.collect()
        
print('AVERAGED AUC :', np.mean(aucs))


Modeling Stage
TRAIN:  [     0      1      2 ... 307505 307506 307508] TEST:  [     9     16     25 ... 307507 307509 307510]
1   Fold AUC : 0.8031452843225432
2   Fold AUC : 0.8032109323723113
3   Fold AUC : 0.8032345241916832
4   Fold AUC : 0.8032450373522547
5   Fold AUC : 0.8032521669256527
Fold AUC : 0.8032247412113762
TRAIN:  [     0      2      3 ... 307508 307509 307510] TEST:  [     1      8     17 ... 307497 307503 307504]
1   Fold AUC : 0.8073915089584248
2   Fold AUC : 0.807404409629424
3   Fold AUC : 0.8073974514785667
4   Fold AUC : 0.8073868197698038
5   Fold AUC : 0.8073813202375268
Fold AUC : 0.8074020002076208
TRAIN:  [     0      1      4 ... 307507 307509 307510] TEST:  [     2      3      6 ... 307494 307501 307508]
1   Fold AUC : 0.8062123778133085
2   Fold AUC : 0.8061983310985235
3   Fold AUC : 0.8061836823307842
4   Fold AUC : 0.8061687663797813
5   Fold AUC : 0.8061537257432553
Fold AUC : 0.8061915767056165
TRAIN:  [     0      1      2 ... 307508 307509 3075

In [15]:
test_preds = pd.DataFrame(test_set).T

In [17]:
y_hat = test_preds.rank(axis=0, method='min').mul(test_preds.shape[1] * [1 / test_preds.shape[1]]).sum(1) / test_preds.shape[0] 
    


y_hat.head(10)


sampl_sub = pd.read_csv('../../data/sample_submission.csv')


sampl_sub['TARGET'] = y_hat.values



sampl_sub.to_csv("ridge-stack-submission.csv", index=False)



sampl_sub.head()



,SK_ID_CURR,TARGET
0,100001,0.363733
1,100005,0.806692
2,100013,0.394059
3,100028,0.380761
4,100038,0.785200
